# Challenge 1: Titanic - Machine Learning From Disaster 

## Huấn luyện mô hình cơ bản (Baseline Model Training)

Các mô hình được huấn luyện trên tập dữ liệu gốc (chưa qua feature engineering), nhằm đánh giá hiệu năng cơ bản ban đầu.

### Khai báo thư viện

In [14]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
import joblib
import random

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, KFold

from warnings import filterwarnings
filterwarnings('ignore')

### Tham số thực nghiệm

In [15]:
params = {}

params["exps_dir"] = "../exps"
params["exp_name"] = "challenge1_titanic_standard"

params["exps_root"] = f'{params["exps_dir"]}/result1_standard'
params["save_dir"] = f'{params["exps_dir"]}/result1_{params["exp_name"]}'

params["data_path"] = f'{params["exps_dir"]}/data/train.xlsx'
params["valid_path"] = f'{params["exps_dir"]}/data/valid.xlsx'

params["k_fold"] = 10
params["random_state"] = 42

random.seed(params["random_state"])
os.environ['PYTHONHASHSEED'] = str(params["random_state"])
np.random.seed(params["random_state"])

### Nạp dữ liệu

In [16]:
df_train = pd.read_excel(f"{params["data_path"]}")
df_valid = pd.read_excel(f"{params["valid_path"]}")

In [17]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,1,1,0.566474,0,0,0.055628,56,2
1,0,2,1,0.283740,0,0,0.025374,147,2
2,0,3,1,0.396833,0,0,0.015469,147,2
3,0,3,1,0.321438,1,0,0.015330,147,2
4,0,3,0,0.070118,4,2,0.061045,147,2


In [18]:
df_valid.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,1,0.346569,1,1,0.029758,147,0
1,0,2,1,0.384267,0,0,0.020495,147,2
2,0,3,1,0.246042,0,0,0.015469,147,2
3,1,2,0,0.070118,0,1,0.064412,147,2
4,1,3,0,0.170646,1,0,0.021942,147,0


### Tách X, y

In [19]:
X_train = df_train.drop("Survived", axis=1)
y_train = df_train["Survived"]
X_valid = df_valid.drop("Survived",axis=1)
y_valid = df_valid["Survived"]

print(f"Train shape: {X_train.shape}, {y_train.shape}")
print(f"Valid shape: {X_valid.shape}, {y_valid.shape}")

Train shape: (712, 8), (712,)
Valid shape: (179, 8), (179,)


In [20]:
kfold = KFold(n_splits=params["k_fold"], shuffle=True, random_state=params["random_state"])
print(f"+ X_train: {len(X_train)}")
for fold, (train_idx, valid_idx) in enumerate(kfold.split(X_train, y_train)):
    print(f'Fold {fold}: ')
    print(f'+ train_idx: {train_idx}')
    print(f'+ valid_idx: {valid_idx}')
    print(f'+ train / valid: {valid_idx}')
    pass

+ X_train: 712
Fold 0: 
+ train_idx: [  0   1   2   3   4   5   6   7   8   9  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  40  41  42  43  44  45  46  47  48  50  51  52  53  55  56  57
  58  59  60  61  62  64  66  67  68  69  70  71  73  74  75  76  78  79
  80  82  83  85  87  88  89  90  91  92  93  94  95  96  97  98  99 100
 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 119
 121 122 123 124 125 126 127 128 129 130 132 133 134 137 138 139 140 141
 142 143 144 146 147 149 150 151 152 153 154 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 197 198 199
 200 201 202 203 204 205 206 207 211 212 213 214 216 217 218 219 220 221
 222 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 240 241
 242 243 244 245 246 248 249 250 251 252 253 254 255 256 258 259 260 261
 262 263 264 2

### Lựa chọn mô hình mặc định

In [21]:
models = {
    ('Logistic Regression', LogisticRegression(random_state=params["random_state"])),
    ('Decision Tree', DecisionTreeClassifier(random_state=params["random_state"])),
    ('Random Forest', RandomForestClassifier(random_state=params["random_state"])),
    ('SVM', SVC(random_state=params["random_state"])),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=params["random_state"])),
    ('XGBoost', XGBClassifier(random_state=params["random_state"]))
}

### Huấn luyện và đánh giá từng mô hình

In [22]:
results = []
baseline_results = {}

for name,model in models:
    baseline_results[name] = {"acc":[],"prec":[],"rec":[],"f1":[]}

    print(f'Model {name}:')

    kfold = KFold(n_splits=params["k_fold"], shuffle=True, random_state=params["random_state"])

    for fold, (train_idx, valid_idx) in enumerate(kfold.split(X_train, y_train)):
        X1_train, y1_train = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X1_valid, y1_valid = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

        model.fit(X1_train,y1_train)

        y_pred_valid = model.predict(X1_valid)

        acc = accuracy_score(y1_valid, y_pred_valid)
        prec = precision_score(y1_valid, y_pred_valid)
        rec = recall_score(y1_valid, y_pred_valid)
        f1 = f1_score(y1_valid, y_pred_valid)

        baseline_results[name]["acc"].append(acc)
        baseline_results[name]["prec"].append(prec)
        baseline_results[name]["rec"].append(rec)
        baseline_results[name]["f1"].append(f1)
        pass

    # hiển thị tham số mô hình
    print(f'+ params = {model.get_params()}')

    # hiển thị kết quả trên từng fold
    print(f'+ acc = {baseline_results[name]["acc"]}')

    # trung bình kết quả +/ std cho tất cả các fold,
    # std: độ lệch chuẩn=> nếu cùng mean thì độ lệch nào nhỏ hơn thì tốt hơn vì ổn định và tập trung hơn
    s_msg = f'+ mean_acc = {np.mean(baseline_results[name]["acc"]):.6f} +/- {np.std(baseline_results[name]["acc"]):.6f}'
    print(s_msg)
    print()

    results.append([
        name,
        np.mean(baseline_results[name]["acc"]),
        np.mean(baseline_results[name]["prec"]),
        np.mean(baseline_results[name]["rec"]),
        np.mean(baseline_results[name]["f1"])
    ])
    #print(classification_report(y1_valid, y_pred_valid, digits=4))

Model Gradient Boosting:
+ params = {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
+ acc = [0.8611111111111112, 0.8333333333333334, 0.8591549295774648, 0.7746478873239436, 0.8028169014084507, 0.8169014084507042, 0.8450704225352113, 0.8169014084507042, 0.8732394366197183, 0.8873239436619719]
+ mean_acc = 0.837050 +/- 0.032956

Model SVM:
+ params = {'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': 42, 'shrinking': True, 'tol': 0.001

In [23]:
df_results = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1"])
df_results = df_results.sort_values(by="F1", ascending=False)
display.display(df_results)

,Model,Accuracy,Precision,Recall,F1
0,Gradient Boosting,0.837050,0.824784,0.709075,0.761776
3,Logistic Regression,0.797750,0.746311,0.693096,0.714517
5,Random Forest,0.794933,0.742044,0.693583,0.714095
2,XGBoost,0.787891,0.735838,0.680712,0.702736
4,Decision Tree,0.779441,0.711446,0.685534,0.695452
1,SVM,0.682727,0.662451,0.310426,0.417568


Nhận xét:
- Mô hình Gradient Boosting đạt kết quả cao nhất trên cả Accuracy (0.837) và F1 (0.762), thể hiện hiệu năng tốt và cân bằng giữa Precision – Recall.
- Logistic Regression và Random Forest có hiệu năng khá ổn định, F1 quanh 0.71, phù hợp làm baseline.
- XGBoost đạt kết quả trung bình khá (F1 ≈ 0.70), cần tinh chỉnh thêm siêu tham số.
- Decision Tree hoạt động ổn định nhưng kém hơn ensemble models.
- SVM có Recall rất thấp (0.31) → mô hình dự đoán thiếu mẫu dương, chưa phù hợp với dữ liệu hiện tại.

=> Gradient Boosting là mô hình tốt nhất, cho hiệu suất tổng thể cao và ổn định nhất trong các mô hình được thử nghiệm.